In [49]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import math

In [93]:
train = pd.read_csv('data/train.csv',parse_dates=[0])
test = pd.read_csv('data/test.csv',parse_dates=[0])

hours = ['h' + str(i) for i in range(0,24)]
months = ['jan', 'feb', 'mar', 'apr', 'may', 'june', 'july', 'aug', 'sept', 'oct', 'nov', 'dec']    
days = ['mon', 'tue', 'wed', 'thur', 'fri', 'sat', 'sun']

set_2 = months + days + hours + ['holiday', 'workingday', 'weather_norm', 'humidity_norm', 'temp_norm', 'windspeed_norm', 'year_norm']
set_1 = ['season','holiday','workingday','weather','humidity','temp','windspeed','hour','month','year', 'day_of_week']

for data_set in [train, test]:

    data_set['day'] = [e.day for e in data_set['datetime']]
    data_set['hour'] = [e.hour for e in data_set['datetime']]
    data_set['month'] = [e.month for e in data_set['datetime']]
    data_set['year'] = [e.year - 2011 for e in data_set['datetime']]
    data_set['day_of_week'] = [e.weekday() for e in data_set['datetime']]
    print data_set[:1]

    def norm(fld_name):
        min_val = min(data_set[fld_name])
        max_val = max(data_set[fld_name])
        diff = max_val - min_val
        data_set[fld_name + '_norm'] = [(e-min_val)/diff for e in data_set[fld_name]]

    for val in set_1:
        norm(val)

    for i, month in enumerate(months):
        data_set[month] = [1 if k == i else 0 for k in data_set['month']]


    for i, day in enumerate(days):
        data_set[day] = [1 if k == i else 0 for k in data_set['day_of_week']]


    for i, hour in enumerate(hours):
        data_set[hour] = [1 if k == i else 0 for k in data_set['hour']]


    datetime  season  holiday  workingday  weather  temp   atemp  humidity  \
0 2011-01-01       1        0           0        1  9.84  14.395        81   

   windspeed  casual  registered  count  day  hour  month  year  day_of_week  
0          0       3          13     16    1     0      1     0            5  
    datetime  season  holiday  workingday  weather   temp   atemp  humidity  \
0 2011-01-20       1        0           1        1  10.66  11.365        56   

   windspeed  day  hour  month  year  day_of_week  
0    26.0027   20     0      1     0            3  


In [94]:
train[:5]['hour']

0    0
1    1
2    2
3    3
4    4
Name: hour, dtype: int64

In [112]:
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')


In [15]:
from sklearn.linear_model import Lasso

In [16]:
clf = Lasso()

In [17]:
clf.fit(train['atemp'],train['count']);
print train['atemp'][0]

14.395


In [18]:
prediction = clf.predict(train['count'])

In [19]:
print prediction

191.574131913


In [47]:
import numpy as np
def error(predictions, actual):
    err = 0
    for p, a in zip(predictions, actual):
        err += (np.log1p(p)-np.log1p(a))**2
    J = (err/len(predictions))**.5
    if math.isnan(J):
        print("debug")
        print(len(predictions))
        print(len(actual))
        print(predictions)
        print(actual)
    return J

err=0
for i in train['count']:
    err += (np.log1p(prediction)-np.log1p(i))**2

print (err/1)**.5
#print error(prediction, train['count'])

163.723867294


In [21]:
import numpy as np
humidity = Lasso()
humidity.fit(train['humidity'],train['count'])
hPredict = humidity.predict(train['count'])
err=0
for i in train['count']:
    err += (np.log1p(hPredict)-np.log1p(i))**2

print (err)**.5

163.723867294


In [22]:
from sklearn import cross_validation
kf = cross_validation.KFold(len(train),5,shuffle=True)
print kf

sklearn.cross_validation.KFold(n=10886, n_folds=5, shuffle=True, random_state=None)


In [129]:
twoFeats = Lasso()
features = ['humidity', 'atemp']
twoFeats.fit(train[features],train['count'])
predict = twoFeats.predict(train[features])
predict[predict < 0] = 0
print error(predict, train['count'])

f = Lasso()
for i, a in enumerate(list(train.columns.values)):
    for j, b in enumerate(list(train.columns.values)[i+1:]):
        for c in list(train.columns.values)[j+2:]:
            print a
            print b
            print c
            features = [a, b, c]
            f.fit(train[features],train['count'])
            p = f.predict(train[features])
            p[ p < 0] = 0
            print('Params: [{}, {}, {}], Error: {}'.format(a, b, c, error(p, train['count'])))
#tdf = pd.DataFrame(data=p, index=test['datetime'], columns=['count'])
#tdf.to_csv('3feat-lasso.csv')




1.39217267465
datetime
season
holiday
['datetime', 'season', 'holiday']


TypeError: float() argument must be a string or a number

In [24]:
print predict

[  63.04235387   59.76404353   59.76404353 ...,  131.82921956  144.00659944
  123.74544336]


In [25]:
error(predict, train['count'])

[  63.04235387   59.76404353   59.76404353   80.02519961   80.02519961
   67.84781973   59.76404353   36.71260254   80.02519961  101.54948508]
[16 40 32 13  1  1  2  3  8 14]


nan

In [26]:
predict[predict < 0] = 0

In [27]:
error(predict, train['count'])

[  63.04235387   59.76404353   59.76404353   80.02519961   80.02519961
   67.84781973   59.76404353   36.71260254   80.02519961  101.54948508]
[16 40 32 13  1  1  2  3  8 14]


1.3921726746499172

In [28]:
threeFeats = Lasso()

In [29]:
predict

array([  63.04235387,   59.76404353,   59.76404353, ...,  131.82921956,
        144.00659944,  123.74544336])

In [30]:
len(predict)

10886

In [31]:
features

['humidity', 'atemp']

In [32]:
test_predictions = twoFeats.predict(test[features])


In [33]:
test_predictions[test_predictions < 0] = 0

In [34]:
tdf = pd.DataFrame([test.index, test_predictions]).transpose()

In [35]:
tdf.columns = ['datetime', 'count']

In [36]:
tdf.to_csv('twoFeats.csv', index=False)


In [80]:
set_1 = ['day', 'atemp']

In [81]:
for i, (train_indexes, test_indexes) in enumerate(kf):
        #model = grid_search.GridSearchCV(model, grid_params, n_jobs=4)
        fit_data = train[set_2].iloc[train_indexes]
        fit_answers = train['count'].iloc[train_indexes]
        check_data = train[set_2].iloc[train_indexes]
        check_answers = train['count'].iloc[train_indexes]
        
        threeFeats.fit(fit_data, fit_answers)

        predictions = threeFeats.predict(check_data)
        predictions = [round(max(0, e)) for e in predictions]
        error_val = error(predictions, check_answers)
        print('Error: {}'.format(error_val))
        
for i, (train_indexes, test_indexes) in enumerate(kf):
        fit_data = train[set_1].iloc[train_indexes]
        fit_answers = train['count'].iloc[train_indexes]
        check_data = train[set_1].iloc[train_indexes]
        check_answers = train['count'].iloc[train_indexes]
        
        threeFeats.fit(fit_data, fit_answers)

        predictions = threeFeats.predict(check_data)
        predictions = [round(max(0, e)) for e in predictions]
        error_val = error(predictions, check_answers)
        print('Test set 1. Error: {}'.format(error_val))

Error: 1.06146844394
Error: 1.05518363451
Error: 1.04966723401
Error: 1.04863387774
Error: 1.0446739249
Test set 1. Error: 1.4506816668
Test set 1. Error: 1.44745679884
Test set 1. Error: 1.45360450878
Test set 1. Error: 1.45183945597
Test set 1. Error: 1.43289562813


In [83]:
normalize = Lasso(fit_intercept=True)
normalize.fit(train[set_2], train['count'])
pred = normalize.predict(test[set_2])
pred[pred < 0] = 0
print pred
print error(pred, test)

tdf = pd.DataFrame(data=pred, index=test['datetime'], columns=['count'])
tdf.to_csv('norm-lasso.csv')

holdout = train[:2177]
rest = train[2177:]

training_error = Lasso()
print rest[:1]
training_error.fit(rest[set_2], rest['count'])
predic = training_error.predict(holdout['count'])
predic[predic < 0] = 0
print error(predic, holdout['count'])


[   0.            0.            0.         ...,  200.26708223  162.72391966
  120.76318032]


TypeError: Not implemented for this type